## Generating Training Set for Stage 1

In [1]:
# import torch
# from Functions_CPTu_Cap_Copy import *
# 
# # Check if CUDA is available
# cuda_available = torch.cuda.is_available()
# print(f"CUDA available: {cuda_available}")
# 
# # If CUDA is available, print out the number of CUDA devices and their names
# if cuda_available:
#     print(f"Number of GPUs available: {torch.cuda.device_count()}")
#     for i in range(torch.cuda.device_count()):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
# else:
#     print("No GPUs detected.")

## Importing CPTu Data


In [1]:
import pandas as pd
import numpy as np
import torch
from Functions_CPTu_Cap_Copy import *
## 
# 
CPTu_data = pd.read_csv('CPTu_train_set.csv', header=None, skiprows=1)
# CPTu_data = pd.read_csv('CPTu_val_set.csv', header=None, skiprows=1)
# CPTu_data = pd.read_csv('CPTu_test_set.csv', header=None, skiprows=1)

print(np.shape(CPTu_data))
print(type(CPTu_data))

(3081, 64)
<class 'pandas.core.frame.DataFrame'>


## Creating L values (Inputs)

In [4]:
l = np.arange(0,30.81,0.1)


# print(type(l))
# print(len(l))
# print(max(l))

# print(CPTu_data[46][100:150])
e_stress = (eff_stress(CPTu_data[18], CPTu_data[19]))
# print(len(e_stress))
# print(type(e_stress))
# print(np.shape(e_stress))
# 
print(skin_frict_calc(4.4, 0.8, e_stress ) + dense_base_cap_calc(4.4, 0.8, e_stress ) )
print(dense_base_cap_calc(4.4, 0.8, e_stress ))

# import numpy as np
# 
# co_eff = 0.04 * np.pi * 0.4 * 0.01
# total_qc_eff_kPa = 3173.47 * 1000  # converting from MPa to kPa
# skin_frict_output = total_qc_eff_kPa * co_eff
# print(skin_frict_output)


404.78985288444164
226.12712681641298


## Implementing Equations for Training Sets

In [4]:
b = 0.4

CPTu_data_np = CPTu_data.to_numpy() 

outputs = S1_bearing_cap(l, b, CPTu_data_np)


    
print(len(outputs))
print(np.shape(outputs))

import math
def replace_nans_with_last_valid(values):
    if not values:
        return values  # Return as is if the list is empty
    
    last_valid = None
    for i in range(len(values)):
        if math.isnan(values[i]):
            if last_valid is not None:
                values[i] = last_valid
        else:
            last_valid = values[i]
    return values

outputs_clean = []
for i in range(CPTu_data.shape[1] // 4):
    temp_output_c = replace_nans_with_last_valid(outputs[i])
    outputs_clean.append(temp_output_c)


print(np.shape(outputs_clean))

16
(16, 309)
(16, 309)


## changing data type

In [7]:
print(outputs[0][:50])

columns_to_drop = [4 * i + j for i in range((CPTu_data.shape[1] // 4)) for j in range(2)]

# Drop the columns
CPTu_data_dropped = CPTu_data.drop(CPTu_data.columns[columns_to_drop], axis=1)

# Flattening the remaining columns for each borehole group
flattened_CPTu_groups = []
for i in range(0, CPTu_data_dropped.shape[1], 2):  # Iterate over each pair of remaining columns
    group = CPTu_data_dropped.iloc[:, i:i+2].values
    flattened_group = np.concatenate(group.T)  # Flatten the group transposed to concatenate by columns
    flattened_CPTu_groups.append(flattened_group)

# Output the shape and type for confirmation
print(np.shape(flattened_CPTu_groups))  # Expected shape: (no_BH, 20) if original was 10 samples per column
print(type(flattened_CPTu_groups))
# print(flattened_CPTu_groups[0])

[125.66370614359174, 83.26226298621032, 103.61748335485063, 121.04828252180177, 142.3812197510265, 152.4814281488921, 173.8943471083566, 208.27680072153035, 257.8990544430255, 328.38889968876373, 349.4518874424544, 352.98703834515607, 391.48642693443844, 411.4420822441913, 423.97459789256254, 428.3472973191409, 412.4961940917525, 380.51185105217087, 377.88501122496876, 391.7571245537558, 369.06147545503364, 359.0013051828731, 353.18235116042996, 342.1838614748832, 335.1610761420102, 334.0777235911523, 339.4498575783365, 347.2908721916705, 353.3128902305209, 364.26999924548556, 369.70270352688203, 380.99538032609144, 393.33555580397103, 406.595599229525, 421.3595550172703, 439.50296096271705, 458.8716836095367, 474.03977868559167, 489.13867401662117, 506.53719849248785, 517.2149618980471, 521.1297455719651, 517.7454209035885, 516.512287686983, 523.5513480212055, 526.4519604275861, 526.2653447819006, 531.1532796765825, 539.0331731667637, 544.6817881738447]
(16, 6162)
<class 'list'>


## Creating CSV file

In [8]:
# Setup the columns for DataFrame
columns = ['Length']  + [f'CPTu_{i}' for i in range(1, 6163)] + ['Bearing_Cap']

# Calculate the total number of rows in the final DataFrame
total_rows =  len(l) * (CPTu_data.shape[1] //4)

# Pre-allocate a numpy array for all data
data_array = np.empty((total_rows, len(columns)))

print(CPTu_data.shape[1] //4)

# Fill the data array
current_row = 0

for i in range(len(l)):
    print(i)
    for j in range(CPTu_data.shape[1] // 4):
        # Each row is Length + Flattened CPTU data + Output
        data_array[current_row, 0] = l[i]
        data_array[current_row, 1:6163] = flattened_CPTu_groups[j] 
        data_array[current_row, 6163] = outputs[j][i]  # Make sure to check index if this is correct
        current_row += 1


# Create a DataFrame from the numpy array
df = pd.DataFrame(data_array, columns=columns)

# Save the DataFrame to a CSV file

# df.to_csv('Stage_1_traind_merged_reduced.csv', index=False)
print("CSV file has been created with the shape:", df.shape)


16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276